In [1]:
# default_exp runs

In [2]:
import sys

In [3]:
sys.path.append('/workspace/oct_ca_seg/oct/oct/')

In [4]:
from oct.startup import *
from model import CapsNet
import numpy as np
import mlflow
from fastai.vision import *
import mlflow.pytorch as MLPY
from fastai.utils.mem import gpu_mem_get_all

In [5]:
gpu_mem_get_all()

[GPUMemory(total=16280, free=16270, used=10),
 GPUMemory(total=16280, free=16270, used=10),
 GPUMemory(total=16280, free=16270, used=10),
 GPUMemory(total=16280, free=16270, used=10)]

### Configuration Setup

In [6]:
name = 'DEEPCAP'

In [7]:
config_dict = loadConfigJSONToDict('configCAPS_small.json')
config_dict['LEARNER']['lr']= 0.001
config_dict['LEARNER']['bs'] = 16
config_dict['LEARNER']['epochs'] = 15
config_dict['LEARNER']['runsave_dir'] = '/workspace/oct_ca_seg/runsaves/'
config_dict['MODEL']['uptype'] = 'deconv' #'upsample'
config_dict['MODEL']['input_images'] = [0,1,2]
config_dict['MODEL']['inputchannels'] = len(config_dict['MODEL']['input_images'])
config = DeepConfig(config_dict)

In [8]:
metrics = [sens, spec, dice, my_Dice_Loss, acc]

In [9]:
def saveConfigRun(dictiontary, run_dir, name):
    with open(run_dir/name, 'w') as file:
        json.dump(dictiontary, file)

## Dataset

In [10]:
cocodata_path = Path('/workspace/oct_ca_seg/COCOdata/')
train_path = cocodata_path/'train/images'
valid_path = cocodata_path/'valid/images'
test_path = cocodata_path/'test/images'

### For complete dataset

In [11]:
def _in_delta(x):
    #print(x.shape)
    return x[config.MODEL.input_images,:,:]
in_delta = TfmPixel(_in_delta, order=0)
in_delta.use_on_y = False

In [12]:
fn_get_y = lambda image_name: Path(image_name).parent.parent/('labels/'+Path(image_name).name)
codes = np.loadtxt(cocodata_path/'codes.txt', dtype=str)
tfms = get_transforms()
tfms[0].append(in_delta())
src = (SegCustomItemList
       .from_folder(cocodata_path, recurse=True, extensions='.jpg')
       .filter_by_func(lambda fname: Path(fname).parent.name == 'images', )
       .split_by_folder('train', 'valid')
       .label_from_func(fn_get_y, classes=codes))
src.transform(tfms, tfm_y=True, size=config.LEARNER.img_size)
data = src.databunch(cocodata_path,
                     bs=config.LEARNER.bs,
                     val_bs=2*config.LEARNER.bs,
                     num_workers = config.LEARNER.num_workers)
stats = data.batch_stats()#[torch.tensor([0.2190, 0.1984, 0.1928]), torch.tensor([0.0645, 0.0473, 0.0434])]
data.normalize(stats);
data.c_in, data.c_out = len(config.MODEL.input_images), 2

In [13]:
data.one_batch()[0].shape

torch.Size([16, 3, 256, 256])

### For converting Validation set into a mini set to experiment on

In [32]:
fn_get_y = lambda image_name: Path(image_name).parent.parent/('labels/'+Path(image_name).name)
codes = np.loadtxt(cocodata_path/'codes.txt', dtype=str)
tfms = get_transforms()
src = (SegCustomItemList
       .from_folder(test_path, recurse=True, extensions='.jpg')
       .filter_by_func(lambda fname: Path(fname).parent.name == 'images', )
       .split_by_rand_pct(0.9)
       .label_from_func(fn_get_y, classes=codes))
src.transform(tfms, tfm_y=True, size =config.LEARNER.img_size)
data = src.databunch(test_path,
                     bs=config.LEARNER.bs,
                     val_bs=2*config.LEARNER.bs,
                     num_workers = config.LEARNER.num_workers)
stats = [torch.tensor([0.2190, 0.1984, 0.1928]), torch.tensor([0.0645, 0.0473, 0.0434])]
data.normalize(stats);
data.c_in, data.c_out = 3, 2

### SEG_MODEL CAPS DECONV CHECK YEAH BOY  

In [21]:
config_dict = loadConfigJSONToDict('configCAPS_small.json')
config_dict['LEARNER']['lr']= 0.001
config_dict['LEARNER']['bs'] = 16
config_dict['LEARNER']['epochs'] = 15
config_dict['LEARNER']['runsave_dir'] = '/workspace/oct_ca_seg/runsaves/'
config_dict['MODEL']['uptype'] = 'deconv' #'upsample'
config_dict['MODEL']['input_images'] = [0,1,2]
config_dict['MODEL']['inputchannels'] = len(config_dict['MODEL']['input_images'])
config = DeepConfig(config_dict)

### SEG_MODEL CAPS DECONV CHECK YEAH BOY  

In [14]:
config_dict = loadConfigJSONToDict('configCAPS_small.json')
config_dict['LEARNER']['lr']= 0.001
config_dict['LEARNER']['bs'] = 16
config_dict['LEARNER']['epochs'] = 15
config_dict['LEARNER']['runsave_dir'] = '/workspace/oct_ca_seg/runsaves/'
config_dict['MODEL']['uptype'] = 'deconv' #'upsample'
config_dict['MODEL']['input_images'] = [0,1]
config_dict['MODEL']['inputchannels'] = len(config_dict['MODEL']['input_images'])
config = DeepConfig(config_dict)

In [15]:
fn_get_y = lambda image_name: Path(image_name).parent.parent/('labels/'+Path(image_name).name)
codes = np.loadtxt(cocodata_path/'codes.txt', dtype=str)
tfms = get_transforms()
tfms[0].append(in_delta())
src = (SegCustomItemList
       .from_folder(cocodata_path, recurse=True, extensions='.jpg')
       .filter_by_func(lambda fname: Path(fname).parent.name == 'images', )
       .split_by_folder('train', 'valid')
       .label_from_func(fn_get_y, classes=codes))
src.transform(tfms, tfm_y=True, size=config.LEARNER.img_size)
data = src.databunch(cocodata_path,
                     bs=config.LEARNER.bs,
                     val_bs=2*config.LEARNER.bs,
                     num_workers = config.LEARNER.num_workers)
stats = data.batch_stats()#[torch.tensor([0.2190, 0.1984, 0.1928]), torch.tensor([0.0645, 0.0473, 0.0434])]
data.normalize(stats);
data.c_in, data.c_out = len(config.MODEL.input_images), 2

In [16]:
config.MODEL.input_images

[0, 1]

In [17]:
deepCap = CapsNet(config.MODEL).cuda()

In [18]:
data.one_batch()[0].shape

torch.Size([16, 2, 256, 256])

In [19]:
deepCap(data.one_batch(cpu=False)[0].cuda());

In [20]:
run_dir = config.LEARNER.runsave_dir+'/'+name
os.makedirs(run_dir, exist_ok=True)
exp_name = 'seg_model_caps'
mlflow_CB = partial(MLFlowTracker,
                    exp_name=exp_name,
                    uri='file:/workspace/oct_ca_seg/runsaves/fastai_experiments/mlruns/',
                    params=config.config_dict,
                    log_model=True,
                    nb_path="/workspace/oct_ca_seg/oct/02_caps.ipynb")

deepCap = CapsNet(config.MODEL).cuda()
learner = Learner(data = data,
                  model = deepCap,
                  metrics = metrics,
                  callback_fns=mlflow_CB)

In [21]:
with mlflow.start_run():
    learner.fit_one_cycle(config.LEARNER.epochs, slice(config.LEARNER.lr), pct_start=0.9)
    MLPY.save_model(learner.model, run_dir+'/model')
    save_all_results(learner, run_dir, exp_name)
    saveConfigRun(config.config_dict, run_dir=Path(run_dir), name = '111configCAPS_APPresnet18_bs16_epochs15_lr0.001.json')

epoch,train_loss,valid_loss,sens,spec,dice,my_Dice_Loss,acc,time


KeyboardInterrupt: 

### SEG_MODEL CAPS UPSAMPLE CHECK YEAH BOY  

In [ ]:
config_dict = loadConfigJSONToDict('configCAPS_small.json')
config_dict['LEARNER']['lr']= 0.001
config_dict['LEARNER']['bs'] = 16
config_dict['LEARNER']['epochs'] = 15
config_dict['LEARNER']['runsave_dir'] = '/workspace/oct_ca_seg/runsaves/'
config_dict['MODEL']['uptype'] = 'upsample' #'upsample'
config_dict['MODEL']['input_images'] = [0,1,2]
config_dict['MODEL']['inputchannels'] = len(config_dict['MODEL']['input_images'])
config = DeepConfig(config_dict)

In [63]:
config.MODEL.input_images

[0, 2]

In [19]:
deepCap = CapsNet(config.MODEL).cuda()

In [20]:
data.one_batch()[0].shape

torch.Size([16, 3, 256, 256])

In [21]:
deepCap(data.one_batch(cpu=False)[0].cuda());

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:2796: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [ ]:
run_dir = config.LEARNER.runsave_dir+'/'+name
os.makedirs(run_dir, exist_ok=True)
exp_name = 'seg_model_caps'
mlflow_CB = partial(MLFlowTracker,
                    exp_name=exp_name,
                    uri='file:/workspace/oct_ca_seg/runsaves/fastai_experiments/mlruns/',
                    params=config.config_dict,
                    log_model=True,
                    nb_path="/workspace/oct_ca_seg/oct/02_caps.ipynb")

deepCap = CapsNet(config.MODEL).cuda()
learner = Learner(data = data,
                  model = deepCap,
                  metrics = metrics,
                  callback_fns=mlflow_CB)

In [ ]:
with mlflow.start_run():
    learner.fit_one_cycle(config.LEARNER.epochs, slice(config.LEARNER.lr), pct_start=0.9)
    MLPY.save_model(learner.model, run_dir+'/model')
    save_all_results(learner, run_dir, exp_name)
    saveConfigRun(config.config_dict, run_dir=Path(run_dir), name = '111upsamples_configCAPS_APPresnet18_bs16_epochs15_lr0.001.json')

### Rest of script

In [43]:
run_dir = config.LEARNER.runsave_dir+'/'+name
os.makedirs(run_dir, exist_ok=True)
exp_name = 'seg_model_caps'
mlflow_CB = partial(MLFlowTracker,
                    exp_name=exp_name,
                    uri='file:/workspace/oct_ca_seg/runsaves/fastai_experiments/mlruns/',
                    params=config.config_dict,
                    log_model=True,
                    nb_path="/workspace/oct_ca_seg/oct/02_caps.ipynb")

deepCap = CapsNet(config.MODEL).cuda()
learner = Learner(data = data,
                  model = deepCap,
                  metrics = metrics,
                  callback_fns=mlflow_CB)

In [46]:
with mlflow.start_run():
    learner.fit_one_cycle(config.LEARNER.epochs, slice(config.LEARNER.lr), pct_start=0.9)
    MLPY.save_model(learner.model, run_dir+'/model')
    save_all_results(learner, run_dir, exp_name)
    saveConfigRun(config.config_dict, run_dir=Path(run_dir), name = 'configCAPS_APPresnet18_bs16_epochs15_lr0.001.json')

epoch,train_loss,valid_loss,sens,spec,dice,my_Dice_Loss,acc,time


torch.Size([16, 4, 64, 64, 16]) 0
torch.Size([16, 2, 64, 64, 4]) 1
torch.Size([16, 2, 32, 32, 4]) 1a
torch.Size([16, 4, 32, 32, 16]) 2
torch.Size([16, 4, 16, 16, 16]) 2a
torch.Size([16, 8, 16, 16, 32]) 3
torch.Size([16, 8, 8, 8, 32]) 3a
torch.Size([16, 8, 8, 8, 32]) bot
Something funny going on with user defined hnew and actual
Something funny going on with user defined wnew and actual


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:2796: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


RuntimeError: The size of tensor a (16) must match the size of tensor b (18) at non-singleton dimension 3

### GIT

In [46]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 4 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   03a_capsAnalysis.ipynb
	modified:   03a_unetAnalysis.ipynb
	deleted:    mlruns/0/0dc64a58b40d4b109b5b46409bc0be53/meta.yaml
	deleted:    mlruns/0/1d6f37b3ef30486f9f5fb555360deb4b/meta.yaml
	deleted:    mlruns/0/23e8384e2e574d898299404b843a990a/meta.yaml
	deleted:    mlruns/0/8cb18c799e6b478f8849e92a24d25a15/meta.yaml
	deleted:    mlruns/0/93329770e7bd4b2bbca4815f499b9764/meta.yaml
	deleted:    mlruns/0/98978ddc3d454c73893cec6e228ca00b/meta.yaml
	deleted:    mlruns/0/b674413753954ede8eb001d145f4c9d0/meta.yaml
	deleted:    mlruns/0/e176dff7437c4ec787fa097d8bae352d/meta.yaml
	deleted:    mlruns/0/e2021677d6ca42b9addeeb9136a78b04/meta.yaml
	deleted:    mlruns/0/meta.yaml
	modified:   model

In [47]:
!git add .

In [48]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 4 commits.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   03a_capsAnalysis.ipynb
	modified:   03a_unetAnalysis.ipynb
	new file:   05_capsUpAndIn.ipynb
	deleted:    mlruns/0/0dc64a58b40d4b109b5b46409bc0be53/meta.yaml
	deleted:    mlruns/0/1d6f37b3ef30486f9f5fb555360deb4b/meta.yaml
	deleted:    mlruns/0/23e8384e2e574d898299404b843a990a/meta.yaml
	deleted:    mlruns/0/8cb18c799e6b478f8849e92a24d25a15/meta.yaml
	deleted:    mlruns/0/93329770e7bd4b2bbca4815f499b9764/meta.yaml
	deleted:    mlruns/0/98978ddc3d454c73893cec6e228ca00b/meta.yaml
	deleted:    mlruns/0/b674413753954ede8eb001d145f4c9d0/meta.yaml
	deleted:    mlruns/0/e176dff7437c4ec787fa097d8bae352d/meta.yaml
	deleted:    mlruns/0/e2021677d6ca42b9addeeb9136a78b04/meta.yaml
	deleted:    mlruns/0/meta.yaml
	modified:   model.py
	new file:   oct_unet4706025.out
	modified:   pawsey/unet.py



In [22]:
!python3 pawsey/caps_01.py

epoch     train_loss  valid_loss  sens      spec      dice      my_Dice_Loss  acc       time    
^Coch 1/10 : |--------------------| 1.13% [4/354 00:05<07:51 0.6947]
Traceback (most recent call last):
  File "pawsey/caps_01.py", line 101, in <module>
    learner.fit_one_cycle(config.LEARNER.epochs, slice(config.LEARNER.lr), pct_start=config.LEARNER.pct_start)
  File "/opt/conda/lib/python3.7/site-packages/fastai/train.py", line 23, in fit_one_cycle
    learn.fit(cyc_len, max_lr, wd=wd, callbacks=callbacks)
  File "/opt/conda/lib/python3.7/site-packages/fastai/basic_train.py", line 200, in fit
    fit(epochs, self, metrics=self.metrics, callbacks=self.callbacks+callbacks)
  File "/opt/conda/lib/python3.7/site-packages/fastai/basic_train.py", line 99, in fit
    for xb,yb in progress_bar(learn.data.train_dl, parent=pbar):
  File "/opt/conda/lib/python3.7/site-packages/fastprogress/fastprogress.py", line 41, in __iter__
    for i,o in enumerate(self.gen):
  File "/opt/conda/lib/python3.7/